# Instructions

*Text Adventure Games* are games in which the player interacts with a rich world only through text. Text adventure games predate computers with graphics. However, in many ways they are more complex than conventional video games because they can involve complicated interactions (e.g., "build a rope bridge") that require a fair amount of imagination. Indeed, text adventure games are used as [research testbeds](https://arxiv.org/abs/1909.05398) for natural language processing agents.

The canonical text adventure game is [Zork](https://en.wikipedia.org/wiki/Zork), in which the player discover an abandoned underworld realm full of treasure. You can find online playable versions.

A text game is made up of individual locations--also called "rooms", though they need not be indoor enclosed spaces as the term might imply. The agent can move between rooms and interact with objects by typing in short commands like "move north" and "take lamp".

In this assignment, we will use a special package that implements text worlds for testing agents: [TextWorld-Express](https://github.com/cognitiveailab/TextWorldExpress). Textworld-Express simplifies text worlds in a few ways: it uses a reduced set of text commands, and rooms laid out in a grid.
TextWorld-Express also implements a few different game objectives, such as cooking, and searching for coins.
TextWorld-Express generates world configurations, so we will need to implement algorithms that are able to complete different game objectives in different world configurations.

In this assignment, our agents will play a custom game wherein the agent must locate a ghost that cannot be directly observed based on noises that can be perceived with some probabilistic uncertainty.

**We will be implementing and using Bayesian Networks for this part of Assignment 3.**

You are prohibited from using any python package that does not come default with Python, except `textworld-express`, `graphviz`, and `pydot`, which are loaded as part of this notebook.

**Notes:**
- If you break execution of a cell running the game engine, you may put TextWorld-Express in an un-recoverable state. If this happens, you will need to reset your kernel/runtime.
- In the Map Reader game, you **can** use the map information.
- ***DO NOT REMOVE ANY COMMENTS THAT HAVE `# export` IN THEM. THE GRADING SCRIPT USES THESE COMMENTS TO EVALUATE YOUR FUNCTIONS. WE WILL NOT AUDIT SUBMISSIONS TO ADD THESE. IF THE AUTOGRADER FAILS TO RUN DUE TO YOUR MODIFICATION OF THESE COMMENTS, YOU WILL NOT RECEIVE CREDIT.***


# Install

Install the TextWorld-Express engine, and graphviz and pydot for visualization.

In [ ]:
%pip install gymnasium
%pip install textworld-express
%pip install graphviz
%pip install pydot

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 742.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.0 MB/s eta 0:00:00


# Imports

In [ ]:
# export
# NOTE: DO NOT MODIFY ANY EXISTING IMPORTS. You are
# allowed to ADD imports from the Python standard
# library.

from textworld_express import TextWorldExpressEnv
import gymnasium
import graphviz
import pydot
import numpy
from IPython.display import Image
from IPython.display import display
import matplotlib.pyplot as plt
from collections import namedtuple
import re
import os
import copy
import json
import math
import random
import networkx as nx
from itertools import combinations

# Load a Game

Set the random seed for repeatablity

In [ ]:
SEED = 3

Initialize the game environment. `ENV` is a global that encapulates the environment.

In [ ]:
ENV = TextWorldExpressEnv(envStepLimit=100)

Set the game generator to generate a particular game (coin game or map reader)

In [ ]:
GAME_TYPE = "coin"
GAME_PARAMS = "numLocations=5,includeDoors=0,numDistractorItems=0"
ENV.load(gameName=GAME_TYPE, gameParams=GAME_PARAMS)

# Ghost Hunting

This environment features a ghost that cannot be seen and might be inside a room or inside the walls (not in a room). However, you can "hear" the ghost make noise. Whenever an action is taken in the world, the agent will also receive information about how far away the ghost sounds. **Unfortunately, this perceived distance is noisy and sometimes it will sound farther away than it actually is, and sometimes it will sound closer than it actually is.**

**Luckily, we also have another tool in our ghost hunting toolbelt**: We are given a special sensor that can give us the horizonal (X coordinate) distance of the ghost from the origin of our grid. The only problem is that this distance is noisy, and we can't see the reproted distance, only a distribution of the different distances.

The agent's goal is to locate where the ghost is.

The `infos` returned by the `env.step()` action returns three extra pieces of information:
- `infos['ghost']` holds a distance (Manhattan distance) to the ghost. This is a "noisy" distance--it may not be the actual distance to the ghost.
- `infos['distribution']` holds a distribution over possible noisy distance values, as a numpy array where the index is the number of units away (Manhattan distance). For example, if `infos['ghost'] = 6` then the distribution will tell you the probability that that value is accurate. If the distribution is:
```
[0.         0.0106383  0.0212766  0.04255319 0.08510638 0.17021277 0.34042553 0.17021277 0.08510638 0.04255319 0.0212766 ]
```
then there is ~34% probability that the ghost is actually 6 units away, but there is ~17% probability that the ghost is actually 5 units away.
- `infos['x_dist_distribution']` holds the distribution of the noisy x distance (**absolute distance**) of the ghost from the origin, as a numpy array where the index is the number of units away (absolute x distance). This operates the same way as the distribution explained above, but for x distance rather than manhattan distance.
- `infos['player']` holds the `(x, y)` position of the player/agent.

**Important: The two sensors are independent. Please note this will effect how you incorporate the two sensors. We highly recommend drawing out the Bayes Diagram/Bayes Rule formula for the sitation as was done in class.**

The agent must guess the `(x, y)` position of the ghost. Once it has a reliable guess, it can perform a special action, `'report'`, which is followed by two numbers separated by spaces that indicate the ghost's x and y position (e.g., `'report 3 4'` would indicate that the agent thought the ghost was in position x=3, y=4). If the guess is correct, the observation returned will be `'True'` and if it is wrong, the observation returned will be `'False'`. Once a report is made, the agent cannot perform any further actions.

In [ ]:
class GhostTextWorldExpressEnv(TextWorldExpressEnv):

  def __init__(self, serverPath=None, envStepLimit=100):
    # Call the super constructor
    super().__init__(serverPath, envStepLimit)
    self.__ghost = None         # Ghost location - (x, y) - initially None
    self.__player = (0, 0)      # Player location - (x, y)
    self.__max_dist = 5         # Range the ghost is allowed to be in. Default is 5
    self.__noise_values = None  # keep track of the +/- actual distance values, relative to the ghost
    self.__noise_counts = None  # Probability (non-normalized) of noise values, relative to the ghost

  ### Override for the environment load function
  def load(self, gameName, gameParams):
    # Call the super method
    super().load(gameName, gameParams)
    # Figure out how many locations are in the gameParams. This will set max_dist
    m = re.search(r'numLocations\=([0-9]+)', gameParams)
    if m is not None and m.lastindex >= 1:
      # numLocations found in gameParams
      self.__max_dist = int(m[1]) + (1 if int(m[1])%2==0 else 0)
    else:
      # Relying on defaults
      self.__max_dist = 5
    # Initialize noise values and noise counts
    self.__noise_values = numpy.array([i - self.__max_dist for i in range((self.__max_dist*2)+1)])
    self.__noise_counts = numpy.array(list(map(lambda x:int(x), [2 ** (self.__max_dist-abs(v)) for v in self.__noise_values])))
    self.__ghost = None
    self.__player = (0, 0)

  ### Override for the environment reset function
  def reset(self, seed=None, gameFold=None, gameName=None, gameParams=None, generateGoldPath=False):
    # Call the super method
    obs, infos = super().reset(seed, gameFold, gameName, gameParams, generateGoldPath)
    # Randomly choose the ghost's location
    while self.__ghost is None or self.__ghost == (0, 0):
        self.__ghost = (random.choice(list(map(lambda x: -1*x, list(range(1,self.__max_dist+1)))) + list(range(self.__max_dist+1))),
                    random.choice(list(map(lambda x: -1*x, list(range(1,self.__max_dist+1)))) + list(range(self.__max_dist+1))))
    # reset the player
    self.__player = (0,0)
    # Compute noisy distance to ghost
    dist = abs(self.__player[0] - self.__ghost[0]) + abs(self.__ghost[1] - self.__player[1]) # Manhattan distance
    noisy_distance = int(max(0, dist + numpy.random.choice(self.__noise_values, p = self.__noise_counts / self.__noise_counts.sum())))
    noisy_distance = min(4 * self.__max_dist, noisy_distance)

    x_dist = abs(0 - self.__ghost[0]) # X distance
    noisy_distance_x = int(max(0, x_dist + numpy.random.choice(self.__noise_values, p = self.__noise_counts / self.__noise_counts.sum())))
    noisy_distance_x = min(4 * self.__max_dist, noisy_distance_x)

    # Make distribution
    infos['distribution'] = self.__make_distribution(noisy_distance)
    infos['x_dist_distribution'] = self.__make_distribution(noisy_distance_x)

    # Add noisy ghost distance to infos
    infos['ghost'] = noisy_distance
    # Add player location to infos
    infos['player'] = self.__player
    # add the 'report' action to the valid actions
    infos['validActions'] = infos['validActions']
    # Add distance info to observation
    if obs == infos['look']:
      infos['look']  = self.__make_ghost_obs(infos['look'], noisy_distance)
      infos['observation'] = infos['look']
      obs = infos['look']
    else:
      infos['look']  = self.__make_ghost_obs(infos['look'], noisy_distance)
    return obs, infos

  def get_max_dist(self):
    return self.__max_dist

  def step(self, action:str):
    # If ghost location is none, then the player cannot perform any actions
    # Check to see if the action is a 'report'
    if action[0:6] == 'report':
      # Player is reporting
      words = action.strip().split()
      if len(words) >= 3:
        x = int(words[1]) # x position being guessed
        y = int(words[2]) # y position being guessed
        if (x, y) == self.__ghost:
          # Guess is correct, report True and terminate game
          self.__ghost = None
          return 'True', 1.0, True, {}
        else:
          # Guess is incorrect, report False and terminate game
          self.__ghost = None
          return 'False', -1.0, True, {}
    # ASSERT: not reporting (or report is ill-formatted)
    if self.__ghost is not None:
      # Call the super method
      observation, reward, isCompleted, infos = super().step(action)
      if observation == infos['look']:
        # when the observation is the same as infos[look], it is because of a location change
        # Change player location (if at all)
        self.__player = self.__change_coordinates(self.__player, action)
      # Compute true distance and noisy distance
      dist = abs(self.__player[0] - self.__ghost[0]) + abs(self.__ghost[1] - self.__player[1]) # Manhattan distance
      noisy_distance = int(max(0, dist + numpy.random.choice(self.__noise_values, p = self.__noise_counts / self.__noise_counts.sum())))
      noisy_distance = min(4 * self.__max_dist, noisy_distance)

      x_dist = abs(0 - self.__ghost[0]) # X distance
      noisy_distance_x = int(max(0, x_dist + numpy.random.choice(self.__noise_values, p = self.__noise_counts / self.__noise_counts.sum())))
      noisy_distance_x = min(4 * self.__max_dist, noisy_distance_x)



      # Make distribution
      infos['distribution'] = self.__make_distribution(noisy_distance)
      infos['x_dist_distribution'] = self.__make_distribution(noisy_distance_x)
      # Add noisy distance to ghost to infos
      infos['ghost'] = noisy_distance
      # Add player location to infos
      infos['player'] = self.__player
      # Add 'report' to valid actions
      infos['validActions'] = infos['validActions']
      infos['observation'] = infos['look']
      if observation == infos['look']:
        # when the observation is the same as infos[look], it is because of a location change
        infos['look'] = self.__make_ghost_obs(infos['look'], noisy_distance)
        observation = infos['look']
      else:
        infos['look'] = self.__make_ghost_obs(infos['look'], noisy_distance)
      # Return all the information
      return observation, reward, isCompleted, infos
    # ASSERT: ghost is not active, don't allow any action to be executed
    return 'The game has ended.', 0.0, True, {}


  ### Make a distribution to share with agent
  def __make_distribution(self, noisy_distance):
    distribution = numpy.zeros(4 * self.__max_dist + 1)
    probs = self.__noise_counts / self.__noise_counts.sum()
    shifted = probs[max(0, self.__max_dist - noisy_distance):min(len(probs), 5 * self.__max_dist - noisy_distance + 1)]
    distribution[max(0, noisy_distance - self.__max_dist):len(shifted)+max(0, noisy_distance - self.__max_dist)] = shifted
    return distribution

  ### Make text about ghost distance to add to observation
  def __make_ghost_obs(self, obs, distance):
    return obs + '\nYou hear a ghost ' + format(distance, ".2f") + ' rooms away.'

  ### Helper function to figure out how player's location changes
  def __change_coordinates(self, coordinates, action):
    if 'move' in action:
      dir = action.split()[1]
      if dir == 'north':
        return (coordinates[0], coordinates[1]+1)
      elif dir == 'south':
        return (coordinates[0], coordinates[1]-1)
      elif dir == 'east':
        return (coordinates[0]+1, coordinates[1])
      elif dir == 'west':
        return (coordinates[0]-1, coordinates[1])
    return coordinates


New environments must be registered through the Gymnasium API.

In [ ]:
gymnasium.register(id='TextWorldExpress-GhostTextWorldExpressEnv-v0',
                   entry_point='__main__:GhostTextWorldExpressEnv')

Create the new environment type.

In [ ]:
ENV = GhostTextWorldExpressEnv(envStepLimit=100)

Create a game with this environment type (same as before)

In [ ]:
GAME_TYPE = "coin"
GAME_PARAMS = "numLocations=5,includeDoors=0,numDistractorItems=0"
ENV.load(gameName=GAME_TYPE, gameParams=GAME_PARAMS)

Rest the environment (same as before).

In [ ]:
obs, infos = ENV.reset(seed=SEED, gameFold="train", generateGoldPath=True)
print(obs)
print('player:', infos['player'])
print('ghost noisy distance:', infos['ghost'])
print('distribution:', infos['distribution'])
print('x distance distribution:', infos['x_dist_distribution'])

Execute a step.

In [ ]:
obs, reward, done, infos = ENV.step('move south')
print(obs)
print('player:', infos['player'])
print('ghost noisy distance:', infos['ghost'])
print('manhattan distribution:', infos['distribution'])
print('x distance distribution:', infos['x_dist_distribution'])

Report a guess (that will be wrong)

In [ ]:
obs, reward, done, infos = ENV.step('report -4 5')
print(obs)
print(reward)
print(done)
print(infos)

False
-1.0
True
{}


## Implement Your Agent

Implement your agent in the `run_agent()` function. The agent takes in an the environment `env` and an alpha value `alpha`. It returns a guess of the ghost's location, which should be the `(x, y)` location where x and y are integers. You will want to report when you have a confidence of at least `alpha`.

The ghost will be located in the box (including the border) defined by the coordinates: (`d`, `d`), (`-d`, `d`), (`-d`, `-d`), (`d`, `-d`). `d` is the max_dist of the environment, which can be retrieved by calling `env.get_max_dist()`.



Use the `env.step()` function to execute actions.

**Do not read private data from the ENV variable, this will result in a 0.**

Interact using only with calling the `env.step()` and `env.get_max_dist()` methods and the results returned from these method calls.

Also, do not use `env.reset()` as this will change the ghost's location to a new, random location.

In [ ]:
# export
# ensure the "export" line is at the top of this cell before submission
def run_agent(env, obs, infos, alpha=0.95):
    d = env.get_max_dist()
    manhattan_dist_distribution = infos['distribution']
    x_dist_distribution = infos['x_dist_distribution']
    player_x, player_y = infos['player']  # Current position of the player
    best_prob = -1
    guess_x, guess_y = 0, 0

    # go thru all x and y
    for x in range(-d, d + 1):
        for y in range(-d, d + 1):
            # get manhattan
            manhattan_dist = abs(player_x - x) + abs(player_y - y)
            # get abs x
            x_dist = abs(x)
            # if distances valid
            if manhattan_dist < len(manhattan_dist_distribution) and x_dist < len(x_dist_distribution):
                # get probability and combine 
                manhattan_prob = manhattan_dist_distribution[manhattan_dist]
                x_prob = x_dist_distribution[x_dist]
                combined_prob = manhattan_prob * x_prob
                # update if this location has the highest probability
                if combined_prob > best_prob:
                    best_prob = combined_prob
                    guess_x, guess_y = x, y

    # if best probabiliy meets confidence level, guess
    if best_prob >= alpha:
        act = f"report {guess_x} {guess_y}"
        obs, _, done, _ = env.step(act)
        print(f"Action: {act}, Observation: {obs}")

    # Return the guessed coordinates
    return guess_x, guess_y



## Testing Suite

In [ ]:
seeds = list(range(5))
environments = [GhostTextWorldExpressEnv(envStepLimit=100)]
games = {'coin':      ['numLocations=5,includeDoors=1,numDistractorItems=0',
                       'numLocations=6,includeDoors=1,numDistractorItems=0',
                       'numLocations=7,includeDoors=1,numDistractorItems=0',
                       'numLocations=10,includeDoors=1,numDistractorItems=0'],
         'mapreader': ['numLocations=5,maxDistanceApart=3,includeDoors=0,maxDistractorItemsPerLocation=0',
                       'numLocations=8,maxDistanceApart=4,includeDoors=0,maxDistractorItemsPerLocation=0',
                       'numLocations=11,maxDistanceApart=5,includeDoors=0,maxDistractorItemsPerLocation=0',
                       'numLocations=15,maxDistanceApart=8,includeDoors=0,maxDistractorItemsPerLocation=0']}

In [ ]:
def test_agent(env, guess_x, guess_y):
  obs, _, _, _ = env.step('report ' + str(guess_x) + ' ' + str(guess_y))
  return obs == 'True'

def run_all(environments, games, seeds, alpha_param=0.95):
  # Results will contain a key (env type, game type, game params, seed) and values will be plans and total_rewards
  results = {}
  # Iterate through all environments given
  for env in environments:
    # set global environment
    ENV = env
    # Iterate through all game types, the keys of the games dict
    for game_type in games:
      # Set the global game type
      GAME_TYPE = game_type
      # Iterate through all game parameters for the given game type in game dict
      for params in games[game_type]:
        # set the global game params
        GAME_PARAMS = params
        # load the environment
        ENV.load(gameName=GAME_TYPE, gameParams=GAME_PARAMS)

        # Iterate through all seeds
        for seed in seeds:
          # set the seed
          obs, infos = ENV.reset(seed)
          x, y = run_agent(ENV, obs, infos, alpha = alpha_param)

          # Store the score in the results
          results[(type(env), game_type, params, seed)] = test_agent(env, x, y)
  return results

In [ ]:
final_results = run_all(environments, games, seeds, alpha_param=0.90)
results = list(final_results.values())
results.count(True) / len(results)

Your agent should get the correct answer on average at least `alpha` percent of the runs. So if alpha is 0.9, your agent should be getting the report correct (written as True in the dictionary `final_results`) on average 90% of the time. **This can be used to verify your solution is working as intended when testing.**

# Grading

Your score for this part of the assignment will be out of **100 points**. We will base your grading based on the ability on your implemention to correctly identify the ghost based on the alpha parameters. We will test your code on hidden seeds and environments with various alpha values, and average your code over multiple different trials to ensure fairness.

There will be a generous "fudge" built into the autograder to account for the randomness inherent in the environment. This ensures that a correct solution should not be penalized for randomness.

Your final score on the hidden environments will be shown on Gradescope when grades are officially published. We will provide sanity checks on a different set of environments/seeds to allow you to verify that your code works as expected in the autograder - ***the sanity checks do not guarantee your code's performance on the hidden tests.***

**We will have checks in place to prevent cheating/shortcuts taken to circumvent the autograder. Being flagged by these checks will lead to further review and a potential 0 on the assignment.**

# Submission

**The due date for this assignment is Wednesday, November 6th, at 11:59 PM EDT.** Upload this notebook with the name `submission.ipynb` file to Gradescope. The autograder will only run successfully if your file is named this way. You must ensure that you have removed all print statements from **your** code, or the autograder may fail to run.

We've added appropriate comments to the top of certain cells for the autograder to export (`# export`). You do NOT have to do anything (e.g. remove print statements) to cells we have provided - anything related to those have been handled for you. You are responsible for ensuring your own code has no syntax errors or unnecessary print statements. You ***CANNOT*** modify the export comments at the top of the cells, or the autograder will fail to run on your submission.

You should ***not*** add any cells to the notebook when submitting. You're welcome to add any code as you need to extra cells when testing, but ***you must remove them when submitting.***

If you identify an issue with the autograder, please feel free to reach out to us on Ed Discussion, or email ygupta46@gatech.edu / rsudhakar9@gatech.edu, with a subject line including "CS 3600".